this notebook make use of 200 samples, train and test them to see if the test result is good.
further, it adds a penalty for negative predictions in the custom MSE loss fn

In [811]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [812]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [813]:
!python3 --version

Python 3.8.10


In [814]:
dataset = np.array([16188101753570999, 16188101953167897, 16188102152819449, 16188102352473247, 16188102552126533, 16188102751777305, 16188102951429619, 16188103151081705, 16188103350734259, 16188103550386827, 16188103750039229, 16188103949691999, 16188104149345511, 16188104348998331, 16188104548651067, 16188104748304685, 16188104947957471, 16188105147615813, 16188105347268801, 16188105546921227, 16188105746579061, 16188105946231987, 16188106145884253, 16188106345543303, 16188106545196109, 16188106744848969, 16188106944507811, 16188107144160627, 16188107343812453, 16188107543470993, 16188107743123977, 16188107942776409, 16188108142434953, 16188108342087759, 16188108541741047, 16188108741399593, 16188108941053265, 16188109140705297, 16188109340363783, 16188109540016723, 16188109739669745, 16188109939329105, 16188110138983137, 16188110338635867, 16188110538294525, 16188110737959433, 16188110937612495, 16188111137264209, 16188111336915637, 16188111536567333, 16188111736218565, 16188111935869963, 16188112135521313, 16188112335173259, 16188112534826793, 16188112734480023, 16188112934132659, 16188113133787133, 16188113333438647, 16188113533091059, 16188113732743981, 16188113932397057, 16188114132049655, 16188114331703189, 16188114531355715, 16188114731008837, 16188114930662207, 16188115130318421, 16188115329973629, 16188115529626341, 16188115729280117, 16188115928934007, 16188116128587121, 16188116328238885, 16188116527892339, 16188116727545689, 16188116927197987, 16188117126850641, 16188117326502411, 16188117526154633, 16188117725813197, 16188117925465183, 16188118125116967, 16188118324775765, 16188118524427667, 16188118724079779, 16188118923738927, 16188119123392053, 16188119323044761, 16188119522702999, 16188119722355261, 16188119922006717, 16188120121669329, 16188120321322889, 16188120520975793, 16188120720635777, 16188120920289981, 16188121119943973, 16188121319603613, 16188121519257537, 16188121718910749, 16188121918570127, 16188122118222727, 16188122317875815, 16188122517535361, 16188122717188877, 16188122916843377, 16188123116502973, 16188123316157393, 16188123515811911, 16188123715465281, 16188123915117041, 16188124114769719, 16188124314421503, 16188124514072381, 16188124713724481, 16188124913376421, 16188125113027773, 16188125312680183, 16188125512331389, 16188125711983233, 16188125911635905, 16188126111287469, 16188126310939273, 16188126510592011, 16188126710243793, 16188126909895347, 16188127109547639, 16188127309199259, 16188127508851201, 16188127708506295, 16188127908156419, 16188128107807471, 16188128307460617, 16188128507113303, 16188128706765687, 16188128906418567, 16188129106071205, 16188129305723885, 16188129505377589, 16188129705031499, 16188129904683723, 16188130104343125, 16188130303996897, 16188130503648715, 16188130703308149, 16188130902960571, 16188131102612853, 16188131302273841, 16188131501928323, 16188131701581079, 16188131901241535, 16188132100894251, 16188132300546873, 16188132500205491, 16188132699857849, 16188132899509511, 16188133099167587, 16188133298819573, 16188133498470873, 16188133698129487, 16188133897782267, 16188134097436949, 16188134297093059, 16188134496745133, 16188134696397649, 16188134896057645, 16188135095709327, 16188135295360735, 16188135495019687, 16188135694671737, 16188135894340379, 16188136093999115, 16188136293651307, 16188136493303213, 16188136692955299, 16188136892606789, 16188137092259521, 16188137291911729, 16188137491563879, 16188137691216859, 16188137890869115, 16188138090521493, 16188138290173819, 16188138489826165, 16188138689478583, 16188138889131841, 16188139088784241, 16188139288436407, 16188139488088987, 16188139687742539, 16188139887394845, 16188140087048267, 16188140286700297, 16188140486352527, 16188140686005187, 16188140885656801, 16188141085308607, 16188141284961245, 16188141484613245])
dataset = dataset * 0.00000001

In [815]:
dataset[0:2]

array([1.61881018e+08, 1.61881020e+08])

In [816]:
dataset.shape

(200,)

In [817]:
start = dataset[0]
end = dataset[-1]
interval = end - start
start, end, interval


(161881017.53571, 161881414.84613246, 397.31042245030403)

In [818]:
dataset = dataset - start
dataset[0], dataset[-1]

(0.0, 397.31042245030403)

In [819]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = dataset.reshape(-1,1)
dataset = scaler.fit_transform(dataset)
dataset

array([[0.        ],
       [0.0050237 ],
       [0.01004878],
       [0.01507391],
       [0.02009903],
       [0.02512409],
       [0.03014919],
       [0.03517428],
       [0.04019938],
       [0.04522448],
       [0.05024958],
       [0.05527469],
       [0.06029982],
       [0.06532492],
       [0.07035003],
       [0.07537516],
       [0.08040027],
       [0.08542552],
       [0.09045063],
       [0.09547573],
       [0.10050096],
       [0.10552608],
       [0.11055117],
       [0.11557644],
       [0.12060155],
       [0.12562666],
       [0.13065192],
       [0.13567703],
       [0.14070211],
       [0.14572736],
       [0.15075248],
       [0.15577757],
       [0.16080283],
       [0.16582794],
       [0.17085306],
       [0.17587831],
       [0.18090344],
       [0.18592853],
       [0.19095378],
       [0.19597889],
       [0.20100401],
       [0.20602928],
       [0.21105442],
       [0.21607953],
       [0.22110478],
       [0.2261302 ],
       [0.23115531],
       [0.236

In [820]:
old_dataset = scaler.inverse_transform(dataset)
old_dataset

array([[  0.        ],
       [  1.99596897],
       [  3.99248448],
       [  5.98902246],
       [  7.98555532],
       [  9.98206303],
       [ 11.9785862 ],
       [ 13.97510704],
       [ 15.9716326 ],
       [ 17.96815827],
       [ 19.96468228],
       [ 21.96120998],
       [ 23.95774511],
       [ 25.95427331],
       [ 27.95080069],
       [ 29.94733682],
       [ 31.9438647 ],
       [ 33.94044811],
       [ 35.93697798],
       [ 37.93350229],
       [ 39.93008059],
       [ 41.92660987],
       [ 43.92313251],
       [ 45.91972303],
       [ 47.91625106],
       [ 49.91277969],
       [ 51.90936813],
       [ 53.90589628],
       [ 55.90241453],
       [ 57.89899993],
       [ 59.89552975],
       [ 61.89205408],
       [ 63.88863951],
       [ 65.8851676 ],
       [ 67.88170049],
       [ 69.87828591],
       [ 71.87482265],
       [ 73.87134296],
       [ 75.86792785],
       [ 77.86445725],
       [ 79.86098742],
       [ 81.85758105],
       [ 83.85412136],
       [ 85

In [821]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train_size,test_size

(134, 66)

In [822]:
look_back = 50
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [823]:
trainX, trainY = create_dataset(train, look_back=look_back)
testX, testY = create_dataset(test, look_back)

In [824]:
trainX[:2],trainY[:2]
testX[:2],testY[:2]

(array([[0.67336623, 0.67839133, 0.68341644, 0.68844155, 0.69346665,
         0.69849178, 0.70351692, 0.70854201, 0.71356729, 0.71859242,
         0.72361751, 0.72864278, 0.73366788, 0.73869298, 0.74371829,
         0.74874344, 0.75376855, 0.75879385, 0.76381896, 0.76884406,
         0.77386932, 0.77889441, 0.78391949, 0.78894473, 0.79396982,
         0.79899489, 0.80402015, 0.80904526, 0.81407041, 0.8190956 ,
         0.82412069, 0.8291458 , 0.83417109, 0.83919617, 0.84422124,
         0.8492465 , 0.85427159, 0.8592971 , 0.86432236, 0.86934745,
         0.87437254, 0.87939763, 0.8844227 , 0.88944781, 0.8944729 ,
         0.899498  , 0.90452311, 0.9095482 , 0.9145733 , 0.9195984 ],
        [0.67839133, 0.68341644, 0.68844155, 0.69346665, 0.69849178,
         0.70351692, 0.70854201, 0.71356729, 0.71859242, 0.72361751,
         0.72864278, 0.73366788, 0.73869298, 0.74371829, 0.74874344,
         0.75376855, 0.75879385, 0.76381896, 0.76884406, 0.77386932,
         0.77889441, 0.78391949, 

In [825]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [826]:
trainX

array([[[0.        , 0.0050237 , 0.01004878, ..., 0.23618039,
         0.24120547, 0.24623055]],

       [[0.0050237 , 0.01004878, 0.01507391, ..., 0.24120547,
         0.24623055, 0.25125562]],

       [[0.01004878, 0.01507391, 0.02009903, ..., 0.24623055,
         0.25125562, 0.25628069]],

       ...,

       [[0.40200914, 0.40703423, 0.41205931, ..., 0.63819057,
         0.64321565, 0.64824074]],

       [[0.40703423, 0.41205931, 0.41708457, ..., 0.64321565,
         0.64824074, 0.65326591]],

       [[0.41205931, 0.41708457, 0.42210966, ..., 0.64824074,
         0.65326591, 0.65829095]]])

In [827]:
trainY

array([0.25125562, 0.25628069, 0.26130576, 0.26633085, 0.27135598,
       0.2763811 , 0.2814062 , 0.28643135, 0.29145643, 0.29648153,
       0.30150664, 0.30653175, 0.31155686, 0.31658198, 0.32160709,
       0.3266322 , 0.33165733, 0.33668252, 0.34170769, 0.3467328 ,
       0.35175793, 0.35678306, 0.36180818, 0.36683326, 0.37185839,
       0.37688351, 0.38190861, 0.38693371, 0.39195879, 0.39698389,
       0.40200914, 0.40703423, 0.41205931, 0.41708457, 0.42210966,
       0.42713475, 0.43216002, 0.43718513, 0.44221024, 0.44723549,
       0.45226058, 0.45728566, 0.46231101, 0.46733614, 0.47236125,
       0.47738654, 0.48241168, 0.48743682, 0.4924621 , 0.49748724,
       0.50251236, 0.50753763, 0.51256274, 0.51758785, 0.52261313,
       0.52763826, 0.53266341, 0.53768869, 0.54271384, 0.54773899,
       0.55276411, 0.55778919, 0.5628143 , 0.56783938, 0.57286444,
       0.57788953, 0.58291462, 0.58793969, 0.59296479, 0.59798986,
       0.60301494, 0.60804005, 0.61306513, 0.61809021, 0.62311

In [828]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    # Calculate the squared error between true and predicted values
    squared_error = K.square(y_true - y_pred)
    
    # Define a penalty factor for negative predictions
    penalty_factor = 1000000
    
    # Apply the penalty factor to negative predictions
    penalized_error = K.switch(y_pred < 0, squared_error * penalty_factor, squared_error)
    
    # Calculate the mean of the penalized errors
    loss = K.mean(penalized_error)
    
    return loss

In [829]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
83/83 - 1s - loss: 20713.0000 - accuracy: 0.0000e+00 - 1s/epoch - 17ms/step
Epoch 2/10
83/83 - 0s - loss: 0.1553 - accuracy: 0.0000e+00 - 120ms/epoch - 1ms/step
Epoch 3/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 123ms/epoch - 1ms/step
Epoch 4/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 119ms/epoch - 1ms/step
Epoch 5/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 114ms/epoch - 1ms/step
Epoch 6/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 120ms/epoch - 1ms/step
Epoch 7/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 115ms/epoch - 1ms/step
Epoch 8/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 126ms/epoch - 2ms/step
Epoch 9/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 114ms/epoch - 1ms/step
Epoch 10/10
83/83 - 0s - loss: 0.1552 - accuracy: 0.0000e+00 - 118ms/epoch - 1ms/step


In [830]:
inference_model = tf.keras.models.clone_model(model)
inference_model

In [831]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

1/1 [==============================] - 0s 20ms/step
Train Score: 156.52 RMSE
Test Score: 343.59 RMSE


In [841]:
trainPredict, trainY

(array([[15.600704],
        [16.10182 ],
        [16.59942 ],
        [17.09336 ],
        [17.583504],
        [18.069725],
        [18.551928],
        [19.029982],
        [19.503784],
        [19.97321 ],
        [20.438147],
        [20.898523],
        [21.354227],
        [21.805166],
        [22.25127 ],
        [22.69243 ],
        [23.12855 ],
        [23.559578],
        [23.985443],
        [24.406042],
        [24.821325],
        [25.231228],
        [25.6357  ],
        [26.034676],
        [26.428102],
        [26.815918],
        [27.198105],
        [27.574583],
        [27.945341],
        [28.310335],
        [28.66954 ],
        [29.022919],
        [29.370455],
        [29.712118],
        [30.047886],
        [30.37774 ],
        [30.701681],
        [31.019707],
        [31.331776],
        [31.637918],
        [31.93813 ],
        [32.232395],
        [32.520733],
        [32.803146],
        [33.079643],
        [33.35022 ],
        [33.614937],
        [33.8

In [832]:
testPredict,testY

(array([[38.54661 ],
        [38.457703],
        [38.366863],
        [38.274128],
        [38.179546],
        [38.08315 ],
        [37.984985],
        [37.885082],
        [37.78344 ],
        [37.68013 ],
        [37.575207],
        [37.468655],
        [37.36052 ],
        [37.250816],
        [37.139626]], dtype=float32),
 array([[367.36255163, 369.35907584, 371.3556084 , 373.35213241,
         375.34865406, 377.34517989, 379.3417154 , 381.33823845,
         383.33477268, 385.33129296, 387.32781526, 389.32434186,
         391.320858  , 393.31737608, 395.31390244]]))

In [833]:
a = testPredict-testY
a.mean()

-343.4691369652748

In [834]:
# testPredict += start
# testPredict

In [835]:
testPredict[11, 0]

37.468655

In [836]:
testPredict *= (1 / 0.00000001)

In [837]:
testPredict[10,0]

3757520600.0

In [838]:
testPredict[11,0]

3746865400.0

In [839]:
testPredict += start

In [840]:
testPredict

array([[4.0165422e+09],
       [4.0076513e+09],
       [3.9985674e+09],
       [3.9892938e+09],
       [3.9798356e+09],
       [3.9701960e+09],
       [3.9603796e+09],
       [3.9503892e+09],
       [3.9402250e+09],
       [3.9298941e+09],
       [3.9194017e+09],
       [3.9087465e+09],
       [3.8979331e+09],
       [3.8869627e+09],
       [3.8758436e+09]], dtype=float32)

creating my custom loss function, with a penalty for negative predictions